In [ ]:
"""
긍정 기사 상위 1048576개, 부정 기사 상위 1048576개를 각각 하나의 파일로 생성한다. 
new_dict를 생성하고 후에는 BERT fine-tuning을 위한 파일이다. 
상위 기사를 따로 추출하는 이유는, 확실히 긍정 또는 부정인 기사를 사용하기 위해서이다. 
전체 성별 기사는 10784446이고 한 엑셀의 최대 row는 1048576로, 
전체 성별 기사의 10%인 1078444와 유사하므로 최대 row를 기준으로 상위 파일을 선별한다.

"""


import glob
import os
import pandas as pd

top = 1048576
version = "knu_score" #"new_dict_score"
qlist = ["여성", "여자+-여성", "남성", '남자+-남성']

df_pos = pd.DataFrame(columns=['title', 'source', 'date', 'text', version, 'query'])
df_neg = pd.DataFrame(columns=['title', 'source', 'date', 'text', version, 'query'])
df_pos_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#3_'+version, f"{version+'_pos'}.xlsx")
df_neg_name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#3_'+version, f"{version+'_neg'}.xlsx")


df_list = []
for query in qlist:
    print(query)
    path = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#3_'+version, '#3_'+query)
    g = map(pd.read_excel, glob.glob(path+"/*.xlsx"))
    d = pd.concat(list(g), ignore_index=True)
    d = d.drop(columns=['url'])
    d['query'] = query
    df_list.append(d)

print('concat')
df = pd.concat(df_list, ignore_index=True)
df_list = list()
print('df_pos')
df_pos = df.nlargest(top, version)
print('df_neg')
df_neg = df.nsmallest(top, version)

print('to_excel')
writer_pos = pd.ExcelWriter(df_pos_name, engine='xlsxwriter')
df_pos.to_excel(writer_pos)
writer_pos.book.use_zip64()
writer_pos.save()

writer_neg = pd.ExcelWriter(df_neg_name, engine='xlsxwriter')
df_neg.to_excel(writer_neg)
writer_neg.book.use_zip64()
writer_neg.save()